# Clasificación bayesiana

El siguiente notebook es explicativo, preferiblemente usar el servidor de la escuela. 

Utilizaremos el dataset del corpus GeniaMK para polaridad, el corpus GENIA-MK consiste en 1000 abstracts de literatura biomédica anotados con eventos de alto nivel de información que llaman metaconocimiento. Se define metaconocimiento como las diferentes formas en que la información relacionada con la interpretación puede expresarse en un texto.  Entre ellos se encuentra la dimensión polaridad que tiene como objetivo el identificar si un evento describe una situación positiva o negativa. La forma más común en que se presentan estos eventos es con el uso de la palabra no (*not o no*) aunque también considera palabras que expresan que el evento no ocurre como *fail, lack, unable, exception, without*. Algunos ejemplos:


*   CsA was found **not** to inhibit lck gene expression, nor the activity of the lck gene product.

*   In contrast, NF-kappa B p50 alone **fails** to stimulate kappa B-directed transcription, and based on prior in vitro studies, is not directly regulated by I kappa B.

Entonces, existen 2 categorías posibles: positivo (el valor predeterminado) y negativo.
El archivo “genia_lemma.txt” contiene todas las oraciones recolectadas del corpus GENIA-MK y el archivo “polarity_classes.txt” contiene la clasificación para cada oración (son archivos pareados: las líneas corresponden uno a uno entre los dos archivos). 

## Naïve Bayes

Naïve Bayes es uno de los algoritmos más simples y poderosos para la clasificación basado en el Teorema de Bayes con una suposición de independencia entre los predictores. Naive Bayes es fácil de construir y particularmente útil para conjuntos de datos muy grandes.

Existen 3 tipos: 
*   Multinomial: clasificador ampliamente utilizado para la clasificación de documentos que mantiene el recuento de palabras frecuentes presentes en los documentos.
*   Bernoulli: se utiliza para datos discretos, donde las características solo están en forma binaria.
*   Gaussiano: se utiliza cuando se trata de datos continuos y se utiliza la distribución gaussiana.

En este documento implementaremos el Bernoulli y el Multinomial.



### BernoulliNB

Vamos a dividir nuestros datos en un conjunto de entrenamiento y uno para probar que tan bien entreno (70/30 respectivamente), después generamos un vectorizador con CountVectorizer()la función fit() permite entrenar el vectorizador con transform() transformamos nuestros datos a un vector binario una vez entrenado el vectorizador y fit_transform entrena devuelve la vectorización. Después de vectorizar podemos  entrenar nuestro clasificador. 

In [3]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,classification_report, make_scorer
import pandas as pd
import numpy as np



In [4]:
#Lectura de archivos de datos
import os

data = pd.read_csv("../data/cvd_transformed.csv")
data.head(6)

,id,age_days,age_year,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi,pulse,bmi_clasification,pulse_clasification
0,0,18393,50.391781,2,168,62.0,110,80,1,1,0,0,1,0,21.967120,30,Peso normal,2
1,1,20228,55.419178,1,156,85.0,140,90,3,1,0,0,1,1,34.927679,50,Obesidad,4
2,2,18857,51.663014,1,165,64.0,130,70,3,1,0,0,0,1,23.507805,60,Peso normal,4
3,3,17623,48.282192,2,169,82.0,150,100,1,1,0,0,1,1,28.710479,50,Sobrepeso,4
4,4,17474,47.873973,1,156,56.0,100,60,1,1,0,0,0,0,23.011177,40,Peso normal,3
5,8,21914,60.038356,1,151,67.0,120,80,2,2,0,0,0,0,29.384676,40,Sobrepeso,3


In [5]:
# Verificar categorías de clase
data['cardio'].unique()

array([0, 1], dtype=int64)

In [6]:
# Verificar dimensionalidad de datos
data.shape

(61839, 18)

In [18]:
# Preparar datos
# X = sólo datos de características de ejemplos
# y = categorías de clase para cada ejemplo
DataSetX = data.iloc[:, np.r_[2, 8:12, 14:15]]
DataSetY = list(data.iloc[:, 13])  # all rows, label only


DataSetX.to_csv('../data/cvd_nocardio.csv', sep=" ", header=False, index=False)

with open(os.path.join("../data/cvd_nocardio.csv"), mode="r") as dFile:
    DataSetX = [line.rstrip() for line in dFile]


print("Datos")
print(DataSetX[:5])
print("Categorias de clase")
print(DataSetY[:5])

Datos
['50.39178082 1 1 0 0 21.9671201814059', '55.41917808 3 1 0 0 34.9276791584484', '51.6630137 3 1 0 0 23.5078053259871', '48.28219178 1 1 0 0 28.7104793249536', '47.8739726 1 1 0 0 23.0111768573307']
Categorias de clase
[0, 1, 1, 1, 0]


In [19]:
#Separamos nuestros datos en training y test
trainX, testX,trainY,testY = train_test_split(DataSetX, DataSetY, test_size = 0.30)
#Creamos un Vectorizador
vectorizer = CountVectorizer()
#entrenamos nuestro vectorizador con nuestros datos de training 
#vectorizamos el test con el mismo vectorizador entrenado para que tengan las mismas dimensiones 
Xtrain = vectorizer.fit_transform(trainX)
Xtest=vectorizer.transform(testX)
#generamos nuestro clasificador
clf = BernoulliNB()
#entrenamos con nuestos modelo
clf.fit(Xtrain, trainY)
#predecimos nuestro dataset de test
Ypred=clf.predict(Xtest)

In [20]:
print('Accuracy: {}\n'.format(accuracy_score(Ypred,testY)))
print('Precision: {}\n'.format(precision_score(Ypred,testY, average='weighted')))
print('Recall: {}\n'.format(recall_score(Ypred,testY, average='weighted')))
print('F-score: {}\n'.format(f1_score(Ypred,testY, average='weighted')))
print('\nConfusion matrix: \n')
print(str(confusion_matrix(Ypred,testY)) + '\n')
print('Classification report: \n')
print(classification_report(Ypred,testY) + '\n')

Accuracy: 0.6055950840879689

Precision: 0.610520916513697

Recall: 0.6055950840879689

F-score: 0.6071188100494818


Confusion matrix: 

[[6516 4025]
 [3292 4719]]

Classification report: 

              precision    recall  f1-score   support

           0       0.66      0.62      0.64     10541
           1       0.54      0.59      0.56      8011

    accuracy                           0.61     18552
   macro avg       0.60      0.60      0.60     18552
weighted avg       0.61      0.61      0.61     18552




Como podemos ver nuestro clasificador no es muy bueno para la clase positiva pero eso sólo es un reflejo del desbalance de clase que tenemos.

Observemos los hiperparámetros de nuestro clasificador.  

In [8]:
clf.get_params()

{'alpha': 1.0,
 'binarize': 0.0,
 'class_prior': None,
 'fit_prior': True,
 'force_alpha': 'warn'}

Podemos buscar mejores hiperparámetros con **RandomizedSearchCV** y **GridSearchCV**

**GridSearchCV**

Búsqueda exhaustiva de valores de parámetros fijos mediante validación cruzada de k-folds.

Los parámetros de este método son:
- el estimador (nuestro clasificador)
- param_grid: un diccionario con los parámetros de nuestro clasificador, en el caso de los BernoulliNB y MultinomialNB solo requiere de alpha
- jobs: indica el número de procesos paralelos que usará, -1 significa que ejecute todos los posibles
- cv (cross-validation) que tiene por defecto 5 folds

In [21]:
from sklearn.model_selection import GridSearchCV
jobs = -1
crossV = 5
alpha=[]
#generamos 100 numeros del 0 al 1 para utilizar como alphas
for i in range(100):
  alpha.append(i/100) 
parameters = {'alpha' : alpha}
#generamos nuestro clasificador
Bernoulli = BernoulliNB()
#generamos nuestro objeto GridSearchCV con nuestro clasificador, verbouse es el numero de mensajes que imprime
clf_1 = GridSearchCV(Bernoulli, parameters,cv=crossV, n_jobs=jobs, verbose=10)
#entrenamos nuestro modelo
clf_1.fit(Xtrain, trainY)
#predecimos
Ypred=clf_1.predict(Xtest)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [24]:
print('Accuracy: {}\n'.format(accuracy_score(Ypred,testY)))
print('Precision: {}\n'.format(precision_score(Ypred,testY, average='weighted')))
print('Recall: {}\n'.format(recall_score(Ypred,testY, average='weighted')))
print('F-score: {}\n'.format(f1_score(Ypred,testY, average='weighted')))
print('\nConfusion matrix: \n')
print(str(confusion_matrix(Ypred,testY)) + '\n')
print('Classification report: \n')
print(classification_report(Ypred,testY) + '\n')
#obtenemos los mejores parametros
best_parameters = clf_1.best_estimator_.get_params()
for params in sorted(best_parameters.keys()):
  print("\t%s: %r\n" % (params, best_parameters[params]))

Accuracy: 0.6055950840879689

Precision: 0.6104989526432661

Recall: 0.6055950840879689

F-score: 0.607112929333288


Confusion matrix: 

[[6515 4024]
 [3293 4720]]

Classification report: 

              precision    recall  f1-score   support

           0       0.66      0.62      0.64     10539
           1       0.54      0.59      0.56      8013

    accuracy                           0.61     18552
   macro avg       0.60      0.60      0.60     18552
weighted avg       0.61      0.61      0.61     18552


	alpha: 0.98

	binarize: 0.0

	class_prior: None

	fit_prior: True

	force_alpha: True



Como podemos ver buscando entre nuestra grid de parámetros encontramos un parámetro que funcionó mejor para clasificar nuestros datos negativos

In [25]:
#obtenemos los parámetros que utilizamos en nuestro modelo
clf_1.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__alpha': 1.0,
 'estimator__binarize': 0.0,
 'estimator__class_prior': None,
 'estimator__fit_prior': True,
 'estimator__force_alpha': True,
 'estimator': BernoulliNB(),
 'n_jobs': -1,
 'param_grid': {'alpha': [0.0,
   0.01,
   0.02,
   0.03,
   0.04,
   0.05,
   0.06,
   0.07,
   0.08,
   0.09,
   0.1,
   0.11,
   0.12,
   0.13,
   0.14,
   0.15,
   0.16,
   0.17,
   0.18,
   0.19,
   0.2,
   0.21,
   0.22,
   0.23,
   0.24,
   0.25,
   0.26,
   0.27,
   0.28,
   0.29,
   0.3,
   0.31,
   0.32,
   0.33,
   0.34,
   0.35,
   0.36,
   0.37,
   0.38,
   0.39,
   0.4,
   0.41,
   0.42,
   0.43,
   0.44,
   0.45,
   0.46,
   0.47,
   0.48,
   0.49,
   0.5,
   0.51,
   0.52,
   0.53,
   0.54,
   0.55,
   0.56,
   0.57,
   0.58,
   0.59,
   0.6,
   0.61,
   0.62,
   0.63,
   0.64,
   0.65,
   0.66,
   0.67,
   0.68,
   0.69,
   0.7,
   0.71,
   0.72,
   0.73,
   0.74,
   0.75,
   0.76,
   0.77,
   0.78,
   0.79,
   0.8,
   0.81,
   0.82,
   0.83,
   

**RandomizedSearchCV**

Búsqueda aleatoria de hiperparámetros.
A diferencia de GridSearchCV, no se prueban todos los valores de los parámetros, sino que se muestrea un número fijo de configuraciones de parámetros a partir de las distribuciones especificadas. Tiene un parámetro adicional a GridSearchCV que es n_iter que especifica el número de iteraciones en búsqueda de los mejor hiperparametros dada una distribución.  

In [26]:
from sklearn.model_selection import RandomizedSearchCV
import scipy
nIter = 100
#generamos una distribucion exponencial para nuestra alpha
paramGrid = {'alpha': scipy.stats.expon(scale=1.0)}
#generamos nuestro clasificador
Bernoulli = BernoulliNB()
#Generamos nuestro objeto RandomizedSearchCV
clf_2 = RandomizedSearchCV(Bernoulli, paramGrid,n_iter=nIter,cv=crossV, n_jobs=jobs, verbose=10)
#entrenamos el modelo
clf_2.fit(Xtrain, trainY)
#Predecimos
Ypred=clf_2.predict(Xtest)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [27]:
print('Accuracy: {}\n'.format(accuracy_score(Ypred,testY)))
print('Precision: {}\n'.format(precision_score(Ypred,testY, average='weighted')))
print('Recall: {}\n'.format(recall_score(Ypred,testY, average='weighted')))
print('F-score: {}\n'.format(f1_score(Ypred,testY, average='weighted')))
print('\nConfusion matrix: \n')
print(str(confusion_matrix(Ypred,testY)) + '\n')
print('Classification report: \n')
print(classification_report(Ypred,testY) + '\n')
best_parameters = clf_2.best_estimator_.get_params()
for params in sorted(best_parameters.keys()):
  print("\t%s: %r\n" % (params, best_parameters[params]))

Accuracy: 0.6097994825355757

Precision: 0.6185791206901197

Recall: 0.6097994825355757

F-score: 0.6123016547803418


Confusion matrix: 

[[6707 4138]
 [3101 4606]]

Classification report: 

              precision    recall  f1-score   support

           0       0.68      0.62      0.65     10845
           1       0.53      0.60      0.56      7707

    accuracy                           0.61     18552
   macro avg       0.61      0.61      0.60     18552
weighted avg       0.62      0.61      0.61     18552


	alpha: 4.492271552091044

	binarize: 0.0

	class_prior: None

	fit_prior: True

	force_alpha: True



### MultinomialNB

In [28]:
from sklearn.naive_bayes import MultinomialNB
paramGrid = {'alpha': scipy.stats.expon(scale=1.0)}
Multinomial = MultinomialNB()
clf_multi = RandomizedSearchCV(Multinomial, paramGrid,n_iter=nIter,cv=crossV, n_jobs=jobs, verbose=10)
clf_multi.fit(Xtrain, trainY)
Ypred=clf_multi.predict(Xtest)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [29]:
print('Accuracy: {}\n'.format(accuracy_score(Ypred,testY)))
print('Precision: {}\n'.format(precision_score(Ypred,testY, average='weighted')))
print('Recall: {}\n'.format(recall_score(Ypred,testY, average='weighted')))
print('F-score: {}\n'.format(f1_score(Ypred,testY, average='weighted')))
print('\nConfusion matrix: \n')
print(str(confusion_matrix(Ypred,testY)) + '\n')
print('Classification report: \n')
print(classification_report(Ypred,testY) + '\n')
best_parameters = clf_multi.best_estimator_.get_params()
for params in sorted(best_parameters.keys()):
  print("\t%s: %r\n" % (params, best_parameters[params]))

Accuracy: 0.6092604570935748

Precision: 0.6175003891246813

Recall: 0.6092604570935748

F-score: 0.6116295081646692


Confusion matrix: 

[[6683 4124]
 [3125 4620]]

Classification report: 

              precision    recall  f1-score   support

           0       0.68      0.62      0.65     10807
           1       0.53      0.60      0.56      7745

    accuracy                           0.61     18552
   macro avg       0.60      0.61      0.60     18552
weighted avg       0.62      0.61      0.61     18552


	alpha: 4.273306215209096

	class_prior: None

	fit_prior: True

	force_alpha: True



In [1]:
! pip install imbalanced-learn 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 1.5 MB/s eta 0:00:00m eta 0:00:010:01m01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 6.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 8.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [21]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(Xtrain, trainY)
print(len(y_resampled))
counts = Counter(y_resampled)
print(counts)

1914
Counter({'Negative': 957, 'Positive': 957})


In [22]:
nIter = 100
#generamos una distribucion exponencial para nuestra alpha
paramGrid = {'alpha': scipy.stats.expon(scale=1.0)}
#generamos nuestro clasificador
Bernoulli = BernoulliNB()
#Generamos nuestro objeto RandomizedSearchCV
clf_under = RandomizedSearchCV(Bernoulli, paramGrid,n_iter=nIter,cv=crossV, n_jobs=jobs, verbose=10)
#entrenamos el modelo
clf_under.fit(X_resampled, y_resampled)
#Predecimos
Ypred=clf_under.predict(Xtest)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [23]:
print('Accuracy: {}\n'.format(accuracy_score(Ypred,testY)))
print('Precision: {}\n'.format(precision_score(Ypred,testY, average='weighted')))
print('Recall: {}\n'.format(recall_score(Ypred,testY, average='weighted')))
print('F-score: {}\n'.format(f1_score(Ypred,testY, average='weighted')))
print('\nConfusion matrix: \n')
print(str(confusion_matrix(Ypred,testY)) + '\n')
print('Classification report: \n')
print(classification_report(Ypred,testY) + '\n')
best_parameters = clf_under.best_estimator_.get_params()
for params in sorted(best_parameters.keys()):
  print("\t%s: %r\n" % (params, best_parameters[params]))

Accuracy: 0.8637980085348507

Precision: 0.8594994738834237

Recall: 0.8637980085348507

F-score: 0.8512513809061895


Confusion matrix: 

[[ 321  313]
 [  70 2108]]

Classification report: 

              precision    recall  f1-score   support

    Negative       0.82      0.51      0.63       634
    Positive       0.87      0.97      0.92      2178

    accuracy                           0.86      2812
   macro avg       0.85      0.74      0.77      2812
weighted avg       0.86      0.86      0.85      2812


	alpha: 3.1981502785952993

	binarize: 0.0

	class_prior: None

	fit_prior: True

	force_alpha: 'warn'

